# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
df = pd.read_csv("Global_weather.csv")
df.dropna(inplace = True)
df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,-37.02,84.0,68.0,1.64,RU,1.610599e+09
1,1,tosya,41.0155,34.0401,3.23,82.0,100.0,1.37,TR,1.610599e+09
2,2,adre,13.4667,22.2000,16.47,30.0,0.0,6.83,TD,1.610599e+09
3,3,kodiak,57.7900,-152.4072,-1.00,86.0,1.0,3.57,US,1.610599e+09
4,4,sault sainte marie,46.5168,-84.3333,0.00,93.0,90.0,2.06,CA,1.610599e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
gmaps.configure(api_key = g_key )


locations = df[["Lat", "Lng"]].astype(float)

hum = df[["Humidity"]].astype(float)
hum = hum["Humidity"].tolist()

In [23]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = hum, 
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
new_df = df[df["Max Temp"] < 30]
new_df = new_df[new_df["Max Temp"] > 25]
new_df = new_df[new_df["Cloudiness"] == 0].reset_index()
new_df.drop(["index", "Unnamed: 0"], axis = 1, inplace = True)
new_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,boa vista,2.8197,-60.6733,26.00,73.0,0.0,5.66,BR,1.610599e+09
1,gold coast,-28.0000,153.4333,29.44,54.0,0.0,6.69,AU,1.610599e+09
2,broken hill,-31.9500,141.4333,29.00,21.0,0.0,6.69,AU,1.610599e+09
3,saint-joseph,-21.3667,55.6167,29.00,70.0,0.0,1.54,RE,1.610599e+09
4,kampot,10.6167,104.1833,28.00,42.0,0.0,4.12,KH,1.610599e+09
5,mazagao,-0.1150,-51.2894,26.00,94.0,0.0,2.06,BR,1.610599e+09
6,ballina,-28.8667,153.5667,29.00,61.0,0.0,5.66,AU,1.610599e+09
7,byron bay,-28.6500,153.6167,28.33,61.0,0.0,5.66,AU,1.610599e+09
8,cayenne,4.9333,-52.3333,26.00,83.0,0.0,2.57,GF,1.610599e+09
9,saint-philippe,-21.3585,55.7679,29.00,70.0,0.0,1.54,RE,1.610599e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
# Create new DataFrame with Hotel Name column
hotel_df = new_df
hotel_df["Hotel Name"] = ""
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,boa vista,2.8197,-60.6733,26.00,73.0,0.0,5.66,BR,1.610599e+09,
1,gold coast,-28.0000,153.4333,29.44,54.0,0.0,6.69,AU,1.610599e+09,
2,broken hill,-31.9500,141.4333,29.00,21.0,0.0,6.69,AU,1.610599e+09,
3,saint-joseph,-21.3667,55.6167,29.00,70.0,0.0,1.54,RE,1.610599e+09,
4,kampot,10.6167,104.1833,28.00,42.0,0.0,4.12,KH,1.610599e+09,
5,mazagao,-0.1150,-51.2894,26.00,94.0,0.0,2.06,BR,1.610599e+09,
6,ballina,-28.8667,153.5667,29.00,61.0,0.0,5.66,AU,1.610599e+09,
7,byron bay,-28.6500,153.6167,28.33,61.0,0.0,5.66,AU,1.610599e+09,
8,cayenne,4.9333,-52.3333,26.00,83.0,0.0,2.57,GF,1.610599e+09,
9,saint-philippe,-21.3585,55.7679,29.00,70.0,0.0,1.54,RE,1.610599e+09,


In [26]:
# Fill Hotel Name column
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    locations = f"{lat},{lng}"
    
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    params = {
    "location": locations,
    "rankby": "distance",
    "type": "lodging",
    "key": g_key,
}
    response = (requests.get(url, params = params)).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except:
        pass
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,boa vista,2.8197,-60.6733,26.00,73.0,0.0,5.66,BR,1.610599e+09,Aipana Plaza Hotel
1,gold coast,-28.0000,153.4333,29.44,54.0,0.0,6.69,AU,1.610599e+09,Gold Coast Holiday Escapes
2,broken hill,-31.9500,141.4333,29.00,21.0,0.0,6.69,AU,1.610599e+09,Excelsior Hotel
3,saint-joseph,-21.3667,55.6167,29.00,70.0,0.0,1.54,RE,1.610599e+09,Maison RUN
4,kampot,10.6167,104.1833,28.00,42.0,0.0,4.12,KH,1.610599e+09,Hak Soun Guesthouse
5,mazagao,-0.1150,-51.2894,26.00,94.0,0.0,2.06,BR,1.610599e+09,Recanto dos Ferreiras
6,ballina,-28.8667,153.5667,29.00,61.0,0.0,5.66,AU,1.610599e+09,Ballina Macnicol Cottage
7,byron bay,-28.6500,153.6167,28.33,61.0,0.0,5.66,AU,1.610599e+09,Tobys Byron Bay - Luxury 6 bedroom holiday house
8,cayenne,4.9333,-52.3333,26.00,83.0,0.0,2.57,GF,1.610599e+09,HOTEL KER ALBERTE
9,saint-philippe,-21.3585,55.7679,29.00,70.0,0.0,1.54,RE,1.610599e+09,"Chambres d'hôte ""La Trinité"""


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,  hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))